## Monthly Accounting

In [16]:
# General Libraries

import os
from bs4 import BeautifulSoup
import requests
import pymongo
import re

In [38]:
# Selenium Connection

from selenium import webdriver
from selenium.webdriver.chrome.service import Service

service = Service(executable_path='r/chromedriver')
options = webdriver.ChromeOptions()
# options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(service=service, options=options)

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# url = "https://www.khanacademy.org/math/probability/xa88397b6:display-quantitative/xa88397b6:frequency-tables-dot-plots/v/frequency-tables-and-dot-plots"
url = "https://track.toggl.com/"
driver.get(url)


In [3]:
# Get Gmail Account and Password

email = input("Google Account:")
password = input("Google Password:")
print('Ok')

Ok


In [40]:
driver.find_element(By.CSS_SELECTOR, '.css-154ozm6-SocialLogin').click()

email_field = WebDriverWait(driver,10).until(
   EC.presence_of_element_located((By.ID, 'identifierId'))
)
email_field.send_keys(email)
email_field.send_keys(Keys.RETURN)

try:

   password_field = WebDriverWait(driver, 10).until(
      # EC.presence_of_element_located((By.CSS_SELECTOR, 'input[type="password"]'))
      EC.presence_of_element_located((By.CSS_SELECTOR, 'input[name="Passwd"]'))
      # https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/#stale-element-reference-exception
      # https://stackoverflow.com/questions/66209119/automation-google-login-with-python-and-selenium-shows-this-browser-or-app-may
   )
   password_field.send_keys(password)
   password_field.send_keys(Keys.RETURN)

except Exception as error:
   print(error)



Message: element not interactable
  (Session info: chrome=115.0.5790.171)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00C5A813+48355]
	(No symbol) [0x00BEC4B1]
	(No symbol) [0x00AF5220]
	(No symbol) [0x00B1D046]
	(No symbol) [0x00B1C8B1]
	(No symbol) [0x00B3A73C]
	(No symbol) [0x00B19A36]
	(No symbol) [0x00B3AA94]
	(No symbol) [0x00B4C922]
	(No symbol) [0x00B3A536]
	(No symbol) [0x00B182DC]
	(No symbol) [0x00B193DD]
	GetHandleVerifier [0x00EBAABD+2539405]
	GetHandleVerifier [0x00EFA78F+2800735]
	GetHandleVerifier [0x00EF456C+2775612]
	GetHandleVerifier [0x00CE51E0+616112]
	(No symbol) [0x00BF5F8C]
	(No symbol) [0x00BF2328]
	(No symbol) [0x00BF240B]
	(No symbol) [0x00BE4FF7]
	BaseThreadInitThunk [0x762400C9+25]
	RtlGetAppContainerNamedObjectPath [0x77517B1E+286]
	RtlGetAppContainerNamedObjectPath [0x77517AEE+238]



In [32]:
password_field = WebDriverWait(driver, 10).until(
   EC.presence_of_element_located((By.CSS_SELECTOR, 'input[type="password"]'))
)
password_field.send_keys(password)
password_field.send_keys(Keys.RETURN)

In [26]:
password_field = WebDriverWait(driver, 10).until(
   EC.presence_of_element_located((By.CSS_SELECTOR, 'input[type="password"]'))
)
password_field.send_keys(password)
email_field.send_keys(Keys.RETURN)

# print(type(password_field))

In [ ]:

# https://stackoverflow.com/questions/8344776/can-selenium-interact-with-an-existing-browser-session
# https://pythonexamples.org/python-selenium-find-element-by-class-name/
# https://www.geeksforgeeks.org/gmail-login-using-python-selenium/
find_element(By.CLASS_NAME, "class_name_value")